In [1]:
# !pip install oolearning --upgrade

In [2]:
import copy
import os
import oolearning as oo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from helpers import column_log, BinaryAucRocScore

pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
width = 10
plt.rcParams['figure.figsize'] = [width, width/1.333]

In [3]:
csv_file = '../census.csv'
target_variable = 'income'

explore = oo.ExploreClassificationDataset.from_csv(csv_file_path=csv_file,
                                                   target_variable=target_variable)
negative_class = '<=50K'
positive_class = '>50K'

explore.dataset.head(20)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
5,37,Private,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
6,49,Private,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.0,0.0,16.0,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
8,31,Private,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
9,42,Private,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K


In [4]:
def create_net_capital(x):
    temp = x.copy()
    temp['net capital'] = temp['capital-gain'] - temp['capital-loss']
    return temp

In [5]:
global_transformations = [oo.ImputationTransformer(),
                          oo.StatelessColumnTransformer(columns=['capital-gain', 'capital-loss'],
                                                        custom_function=column_log),
                          oo.StatelessTransformer(custom_function=create_net_capital),
                          oo.CenterScaleTransformer(),
                          oo.DummyEncodeTransformer(oo.CategoricalEncoding.ONE_HOT)]

In [6]:
params_dict = dict(
#    max_depth=[3, 4],
#    min_child_weight=5,
#    gamma=[0, 0.15, 0.3],
    #subsample=[0.99, 1.0],
    #colsample_bytree=[0.30, 0.35, 0.40, 0.45, 0.5, 0.55],
   # reg_alpha=[0, 0.01, 0.25, 0.05],
   # reg_lambda=[1.5, 2, 2.5],
    #scale_pos_weight=scale_pos_weight_calc,
    learning_rate=[0.04, 0.05, 0.06, 0.07],
    n_estimators=[2500, 3000, 3500],

)
grid = oo.HyperParamsGrid(params_dict=params_dict)
grid.params_grid

,learning_rate,n_estimators
0,0.04,2500
1,0.04,3000
2,0.04,3500
3,0.05,2500
4,0.05,3000
5,0.05,3500
6,0.06,2500
7,0.06,3000
8,0.06,3500
9,0.07,2500


In [7]:
n_positive = np.sum(explore.dataset[target_variable] == positive_class)
n_negative = np.sum(explore.dataset.income == negative_class)
scale_pos_weight_calc = n_negative / n_positive
scale_pos_weight_calc

3.034796573875803

In [8]:
hyper_param_object = oo.XGBoostTreeHP(
                                objective=oo.XGBObjective.BINARY_LOGISTIC,
                                learning_rate=0.1,
                                n_estimators=1000,
                                max_depth=3,
                                min_child_weight=5,
                                gamma=0.15,
                                subsample=1,
                                colsample_bytree=0.4,
                                reg_alpha=0,
                                reg_lambda=2,
                                scale_pos_weight=scale_pos_weight_calc,)

In [9]:
model=oo.XGBoostClassifier()

In [10]:
repeats = 5
folds = 5

In [11]:
model_cache_directory = 'tuner_XGB'
resampler_cache_directory = 'tuner_XGB_resample_cache'

In [12]:
score_list = [oo.AucRocScore(positive_class=positive_class),
              oo.FBetaScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                    positive_class=positive_class),
                            beta=0.5),
              oo.SensitivityScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                          positive_class=positive_class)),
              oo.PositivePredictiveValueScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                                      positive_class=positive_class))]

In [13]:

# define/configure the resampler
resampler = oo.RepeatedCrossValidationResampler(model=model,  # using a Random Forest model
                                             transformations=global_transformations,
                                             scores=[s.clone() for s in score_list],
                                             folds=folds,
                                             repeats=repeats)
# define/configure the ModelTuner
tuner = oo.ModelTuner(resampler=resampler,
                      hyper_param_object=hyper_param_object,
                      model_persistence_manager=oo.LocalCacheManager(cache_directory=model_cache_directory),
                      resampler_persistence_manager=oo.LocalCacheManager(cache_directory=resampler_cache_directory),
                      parallelization_cores=-1,
                      #parallelization_cores=0,
                     )

In [ ]:
tuner.tune(data_x=explore.dataset.drop(columns=target_variable),
           data_y=explore.dataset[target_variable],
           params_grid=grid)

In [ ]:
tuner.results.best_hyper_params

In [ ]:
params_dict

In [ ]:
tuner.results.best_model_resampler_object.score_means

In [ ]:
tuner.results.plot_resampled_stats()

In [ ]:
tuner.results.plot_resampled_scores(metric=oo.Metric.AUC_ROC,
                                    x_axis_limits=(0.9225, 0.935),
                                    show_one_ste_rule=True
                                   )

In [20]:
tuner.results.resampled_stats

,subsample,colsample_bytree,AUC_ROC_mean,AUC_ROC_st_dev,AUC_ROC_cv,F_BETA_mean,F_BETA_st_dev,F_BETA_cv,sensitivity_mean,sensitivity_st_dev,sensitivity_cv,positive_predictive_value_mean,positive_predictive_value_st_dev,positive_predictive_value_cv
0,0.99,0.30,0.929109,0.002511,0.0,0.654190,0.006827,0.01,0.867087,0.006291,0.01,0.616369,0.007284,0.01
1,0.99,0.35,0.929109,0.002458,0.0,0.654154,0.006655,0.01,0.866360,0.006499,0.01,0.616421,0.007120,0.01
2,0.99,0.40,0.929025,0.002503,0.0,0.653663,0.006936,0.01,0.867047,0.006500,0.01,0.615791,0.007437,0.01
3,0.99,0.45,0.928970,0.002438,0.0,0.653629,0.006519,0.01,0.866704,0.006597,0.01,0.615795,0.006989,0.01
4,0.99,0.50,0.928935,0.002478,0.0,0.653674,0.006545,0.01,0.865814,0.006666,0.01,0.615958,0.007015,0.01
5,0.99,0.55,0.928887,0.002384,0.0,0.653435,0.006597,0.01,0.865842,0.006592,0.01,0.615689,0.007079,0.01
6,1.00,0.30,0.929155,0.002474,0.0,0.654405,0.007001,0.01,0.866612,0.007023,0.01,0.616670,0.007546,0.01
7,1.00,0.35,0.929129,0.002506,0.0,0.653861,0.006292,0.01,0.866903,0.006477,0.01,0.616027,0.006755,0.01
8,1.00,0.40,0.929177,0.002508,0.0,0.653979,0.006620,0.01,0.867540,0.006682,0.01,0.616080,0.007167,0.01
9,1.00,0.45,0.929088,0.002413,0.0,0.653613,0.006609,0.01,0.867241,0.006781,0.01,0.615711,0.007101,0.01


In [ ]:
tuner.results.plot_resampled_scores(oo.Metric.FBETA_SCORE,
                                    x_axis_limits=(0.64, 0.67),
                                    show_one_ste_rule=True
                                   )

In [ ]:
params_dict

In [ ]:
tuner.results.plot_hyper_params_profile(
                                        metric=oo.Metric.AUC_ROC,
                                        x_axis='learning_rate',
                                        line='n_estimators',
                                       # grid='gamma'
                                       )

In [ ]:
tuner.results.best_hyper_params

In [ ]:
tuner.results.plot_hyper_params_profile(
                                        metric=oo.Metric.FBETA_SCORE,
                                        x_axis='n_estimators',
                                        line='learning_rate',
                                       # grid='gamma'
                                       )

In [ ]:
tuner.results.plot_hyper_params_profile(
                                        metric=oo.Metric.AUC_ROC,
                                        x_axis='gamma',
                                        line='colsample_bytree',
                                        grid='subsample'
                                       )

In [ ]:
tuner.results.plot_hyper_params_profile(metric=oo.Metric.AUC_ROC,
                                        x_axis='max_depth',
                                        line='min_child_weight',
                                        grid='subsample')

In [ ]:
tuner.results.best_hyper_params

In [ ]:
assert False

# Final Train

In [ ]:
final_hyper_param_object = 



final_model = oo.XGBoostClassifier()
final_transformations = [
                         # kaggle test file has white space around values
                         oo.StatelessColumnTransformer(columns=explore.categoric_features,
                                                       custom_function=lambda x: x.str.strip()),
                         oo.ImputationTransformer(),
                         oo.StatelessColumnTransformer(columns=['capital-gain', 'capital-loss'],
                                                       custom_function=column_log),
                         oo.StatelessTransformer(custom_function=create_net_capital),
                         oo.CenterScaleTransformer(),
                         oo.DummyEncodeTransformer(oo.CategoricalEncoding.ONE_HOT)
]

In [ ]:
evaluator = oo.TwoClassProbabilityEvaluator(converter=
                    oo.TwoClassThresholdConverter(positive_class=positive_class))

trainer = oo.ModelTrainer(model=final_model.clone(),
                          model_transformations=[t.clone() for t in final_transformations],
                          splitter=None,  # don't split, train on all data
                          evaluator=evaluator,
                          scores=[s.clone() for s in score_list])
trainer.train(data=explore.dataset, target_variable=target_variable, hyper_params=final_hyper_param_object.clone())

trainer.training_evaluator.all_quality_metrics

In [ ]:
trainer.training_scores[0].value

In [ ]:
trainer.training_evaluator.plot_roc_curve()

In [ ]:
trainer.training_evaluator.plot_calibration()

In [ ]:
csv_file = '../test_census.csv'
test_dataset = pd.read_csv(csv_file)
test_dataset.shape

In [ ]:
test_dataset.head(100)

In [ ]:
indexes = test_dataset['Unnamed: 0']

predictions = trainer.predict(test_dataset.drop(columns='Unnamed: 0'))
test_converter = oo.TwoClassThresholdConverter(threshold=ideal_threshold,  # ideal thresholds
                                               positive_class=positive_class)
predictions.head()

In [ ]:
class_predictions = test_converter.convert(predictions)
class_predictions[0:5]

In [ ]:
# previous_predictions = pd.read_csv('submission_5_xgb.csv')
# previous_predictions.head(10)

In [ ]:
income_value = predictions['>50K']#[0 if x == '<=50K' else 1 for x in class_predictions]
new_predictions = pd.DataFrame({'id': indexes, 'income': income_value})
new_predictions.to_csv('submission_8_xgb.csv', index=False)

In [ ]:
# # e.g. lower threshold will change 0's to 1's so new - previous will give number predictions that changed to 1
# difference = new_predictions.income - previous_predictions.income
# difference.sum() / len(difference)

# Results

## 2)

Kaggle Public: `0.85769`

File: `submission_5_xgb`

ROC Threshold: `0.51`

Parameters:

```
objective=oo.XGBObjective.BINARY_LOGISTIC,
learning_rate=0.05,
n_estimators=1750,
max_depth=4,
min_child_weight=2,
gamma=0.1,
subsample=1.0,
colsample_bytree=0.15,
reg_alpha=0.005,
scale_pos_weight=scale_pos_weight_calc
```

Originally had Mean Resampler Results of:

```
{'BINARY_AUC': 0.8457153054881816,
 'AUC_ROC': 0.9297213921879113,
 'F_BETA': 0.6565762775126679,
 'sensitivity': 0.8673808546152647,
 'positive_predictive_value': 0.6189823694407532}
```


## 3)

Kaggle Public: `0.85864`

File: `submission_6_xgb`

ROC Threshold: `0.49`

Parameters:

```
objective=oo.XGBObjective.BINARY_LOGISTIC,
learning_rate=0.05,
n_estimators=1750,
max_depth=4,
min_child_weight=2,
gamma=0.1,
subsample=1.0,
colsample_bytree=0.15,
reg_alpha=0.005,
scale_pos_weight=scale_pos_weight_calc
```

Originally had Mean Resampler Results of:

```
{'BINARY_AUC': 0.8457153054881816,
 'AUC_ROC': 0.9297213921879113,
 'F_BETA': 0.6565762775126679,
 'sensitivity': 0.8673808546152647,
 'positive_predictive_value': 0.6189823694407532}
```


----

**NOTE: I realized it doesn't make sense to use 0's and 1's with AUC so I submitted raw probabilities and my score shot up.**

## 4) Using Probabilities

Kaggle Public: `0.93693`

File: `submission_7_xgb`

Parameters:

```
objective=oo.XGBObjective.BINARY_LOGISTIC,
learning_rate=0.05,
n_estimators=2000,
max_depth=3,
min_child_weight=2,
gamma=0.1,
subsample=0.95,
colsample_bytree=0.15,
reg_alpha=0.01,
reg_lambda=2,
scale_pos_weight=scale_pos_weight_calc,
```

Originally had Mean Resampler Results of:

```
{'BINARY_AUC': 0.8452782933202593,
 'AUC_ROC': 0.9292918569630045,
 'F_BETA': 0.655544556494659,
 'sensitivity': 0.8673600361173268,
 'positive_predictive_value': 0.6178407865646417}
```


## 5) Using Probabilities

Kaggle Public: `0.`

File: `submission_9_xgb.csv`

Parameters:

```
...
```

Originally had Mean Resampler Results of:

```
...
```
